In [44]:
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm


In [74]:
main_dir = Path(r"D:\P05_3DRIG_YE-LP\e01_mouse_hunting\lighting_pose_project_2025-06-08\labeled-data")

In [75]:
dirs = [dir for dir in main_dir.iterdir() if dir.is_dir() and dir.name.endswith("permutation")]

In [76]:
def fix_csv(csv_path:Path, vid_name:Path)-> pd.DataFrame:

    csv_broken = pd.read_csv(csv_path, header=[0, 1, 2])



    csv_broken.drop(columns=csv_broken.columns[0], inplace=True)
    # change values second col  with vid_name
    csv_broken.iloc[:, 1] = vid_name


    new_cols = list(csv_broken.columns)

    # Replace the ones at positions 1 and 2 with empty string tuples
    for i in range(1, 3):
        new_cols[i] = tuple("" for _ in new_cols[i])  # Replace each level with empty string

    # Set back the updated columns
    csv_broken.columns = pd.MultiIndex.from_tuples(new_cols)
    return csv_broken



In [77]:
vid_dir = main_dir.parent / "videos"
vid_names = [vid.name.removesuffix('.mp4') for vid in vid_dir.iterdir() if vid.is_file() and vid.name.endswith("permutation.mp4")]

In [78]:
for dir, vid in tqdm(zip(dirs, vid_names), desc="Processing directories"):
    csv_file_path = dir / "CollectedData.csv"
    if csv_file_path.exists():
        try:
            fixed_csv = fix_csv(csv_file_path, vid)
            fixed_csv.to_csv(csv_file_path, index=False)
            fixed_csv.to_hdf(csv_file_path.with_suffix('.h5'), mode='w', key='data', format='table')
        except Exception as e:
            print(f"Error processing {csv_file_path}: {e}")
    else:
        print(f"File {csv_file_path} does not exist, skipping.")
    

Processing directories: 15it [00:00, 39.21it/s]
